In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [2]:
testDF=pd.read_csv('testDF.csv')
testDF

,id,Age,Annual_Premium,Vintage,Response,Response_y,Response_y.1,Response_y.2,Response_y.3,Response_y.4,Response_y.5,Response_y.6
0,11504798,0.235294,0.004869,0.762542,NaN,0.103297,0.041404,0.004198,0.123132,0.228493,0.083005,0.020696
1,11504799,0.552941,0.069392,0.411371,NaN,0.139688,0.177725,0.240530,0.123132,0.228493,0.191811,0.192496
2,11504800,0.552941,0.004869,0.906355,NaN,0.139688,0.177725,0.240530,0.123132,0.228493,0.090684,0.207763
3,11504801,0.258824,0.045360,0.384615,NaN,0.103297,0.041404,0.004198,0.123132,0.000638,0.083005,0.026222
4,11504802,0.600000,0.063157,0.494983,NaN,0.139688,0.177725,0.004198,0.123132,0.228493,0.166799,0.192496
...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,0.670588,0.095639,0.364548,NaN,0.139688,0.177725,0.240530,0.123132,0.228493,0.191811,0.192496
7669862,19174660,0.329412,0.047487,0.615385,NaN,0.139688,0.041404,0.004198,0.123132,0.000638,0.063979,0.026222
7669863,19174661,0.552941,0.004869,0.210702,NaN,0.139688,0.177725,0.004198,0.123132,0.000638,0.095341,0.097062
7669864,19174662,0.352941,0.071952,0.397993,NaN,0.139688,0.041404,0.240530,0.123132,0.228493,0.191811,0.192496


In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [4]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame):
        self.id = df['id'].values
        df= df.drop(columns=['id'])

        self.response = df['Response'].values
        df= df.drop(columns=['Response'])
            
        self.data = df.astype(float).values
        
        pass

    def __len__(self):
        return len(self.id)

    def __getitem__(self, idx):
        x=self.data[idx]
        x=torch.FloatTensor(x)
        x=x.to(device)

        id = self.id[idx]

        y=self.response[idx]
        if y ==0:
            y=[0,1]
        else:
            y=[1,0]
        y=torch.FloatTensor(y)
        y=y.to(device)

        return id, x, y

testSet= MyDataset(testDF)
testSet[0]

(11504798,
 tensor([0.2353, 0.0049, 0.7625, 0.1033, 0.0414, 0.0042, 0.1231, 0.2285, 0.0830,
         0.0207], device='cuda:0'),
 tensor([1., 0.], device='cuda:0'))

In [5]:
testLoader=torch.utils.data.DataLoader(testSet,batch_size=1024)
testLoader

In [6]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convStack = torch.nn.Sequential(
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 2)
        )

    def forward(self, x):
        logits = self.convStack(x)
        return logits

model=torch.load('model.pt')
model

Net(
  (convStack): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [22]:
model.eval()
id_list = []
response_list = []
with torch.no_grad():
    for id, X, _ in testLoader:
        pred = model(X)
        id_list.extend(id.tolist())
        response_list.extend(pred.argmax(dim=1).tolist())
       
response_list

[0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,


In [25]:
resultDF = pd.DataFrame()
resultDF['id'] = pd.DataFrame(id_list)
resultDF['Response'] = pd.DataFrame(response_list)
resultDF

,id,Response
0,11504798,0
1,11504799,1
2,11504800,1
3,11504801,0
4,11504802,0
...,...,...
7669861,19174659,1
7669862,19174660,0
7669863,19174661,0
7669864,19174662,1


In [26]:
resultDF.to_csv('result.csv',index=False)